In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [63]:
def get_address(address: str, kostil=0):
    token = '7cd5ebd6-2115-4662-b850-e3a57c2850c3'
    url = 'http://geocode-maps.yandex.ru/1.x/?apikey=' + token + '&geocode=' + address + '&format=xml&lang=ru_RU&results=1'
    
    
    try:
        
        re = requests.get(url)
        soup = BeautifulSoup(re.text, 'html.parser')

        latitude_longtitude = soup.findAll('pos', attrs=None)
        lat_lon = str(latitude_longtitude).split('<pos>')[1].split('</pos>')[0]
        
        name = soup.findAll('component', attrs=None)

        if 'country' in str(name):
            country = str(name).split('<kind>country</kind><name>')[1].split('</name>')[0]
        else:
            country = 'Неизвестная country'

        if 'province' in str(name):    
            province_1 = str(name).split('<kind>province</kind><name>')[1].split('</name>')[0]
            try:
                province_2 = str(name).split('<kind>province</kind><name>')[2].split('</name>')[0]
            except:
                province_2 = 'Неизвестная province_2'
        else:
            province_1 = 'Неизвестная province_1'
            province_2 = 'Неизвестная province_2'

        if 'area' in str(name):
            area = str(name).split('<kind>area</kind><name>')[1].split('</name>')[0]
        else:
            area = 'Неизвестная area'

        if 'locality' in str(name):
            locality = str(name).split('<kind>locality</kind><name>')[1].split('</name>')[0]
        else:
            locality = 'Неизвестная locality'

        if 'street' in str(name):
            street = str(name).split('<kind>street</kind><name>')[1].split('</name>')[0]
        else:
            locality = 'Неизвестная street'

        if 'house' in str(name):
            house = str(name).split('<kind>house</kind><name>')[1].split('</name>')[0]
        else:
            house = 'Неизвестная house'


        df = pd.DataFrame([[lat_lon.split(' ')[0], 
                            lat_lon.split(' ')[1],
                            country, 
                            province_1, 
                            province_2, 
                            area, locality, 
                            street, 
                            house]])
        df.columns = ['latitude', 
                      'longtitude', 
                      'country', 
                      'province_1', 
                      'province_2', 
                      'area', 'locality',
                      'street', 
                      'house']
        #
        return df
    except:
        if kostil == 0:
            #print(kostil)
            kostil += 1
            reverse_coordinates = (address.split(' ')[1] + '; ' + address.split(' ')[0]).replace(',', '')
            #print(reverse_coordinates)
            return get_address(reverse_coordinates, kostil)
        else:
            return 1

In [64]:
get_address('улица Большая Полянка, 26с2')

,latitude,longtitude,country,province_1,province_2,area,locality,street,house
0,37.618866,55.73698,Россия,Центральный федеральный округ,Москва,Неизвестная area,Москва,улица Большая Полянка,26с2


In [65]:
get_address('Солнечногорск улица Дзержинского, 19')

,latitude,longtitude,country,province_1,province_2,area,locality,street,house
0,36.983604,56.185763,Россия,Центральный федеральный округ,Московская область,городской округ Солнечногорск,Солнечногорск,улица Дзержинского,19


In [66]:
get_address('Заречная улица деревня Ручьи, Конаковский район, Тверская область, Россия')

,latitude,longtitude,country,province_1,province_2,area,locality,street,house
0,36.97401,56.590752,Россия,Центральный федеральный округ,Тверская область,Конаковский район,деревня Ручьи,Заречная улица,Неизвестная house


In [67]:
get_address('30.313263; 59.930317')

,latitude,longtitude,country,province_1,province_2,area,locality,street,house
0,30.313263,59.930317,Россия,Северо-Западный федеральный округ,Санкт-Петербург,Неизвестная area,Санкт-Петербург,переулок Гривцова,5В


In [68]:
get_address('59.930317, 30.313263')

,latitude,longtitude,country,province_1,province_2,area,locality,street,house
0,30.313263,59.930317,Россия,Северо-Западный федеральный округ,Санкт-Петербург,Неизвестная area,Санкт-Петербург,переулок Гривцова,5В


In [114]:
def get_address_json(address: str, kostil=0):
    token = '7cd5ebd6-2115-4662-b850-e3a57c2850c3'
    url_json = 'http://geocode-maps.yandex.ru/1.x/?apikey=' + token + '&geocode=' + address + '&format=json&lang=ru_RU&results=1'
    

    
    re_json = requests.get(url_json)
    json = re_json.json()

    d = dict()
    d['address'] = json['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['metaDataProperty']['GeocoderMetaData']['Address']['formatted']
    d['lat'] = json['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split(' ')[1]
    d['lon'] = json['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split(' ')[0]

    details = json['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['metaDataProperty']['GeocoderMetaData']['Address']['Components']
    for i in details:
        #print(i)
        d[i['kind']] = i['name']

    d['precision'] = json['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['metaDataProperty']['GeocoderMetaData']['precision']


    return pd.DataFrame(data=d, columns=[col for col in d], index=[0])
    

In [115]:
get_address_json('улица Большая Полянка, 26с2')

,address,lat,lon,country,province,locality,street,house,precision
0,"Россия, Москва, улица Большая Полянка, 26с2",55.73698,37.618866,Россия,Москва,Москва,улица Большая Полянка,26с2,exact


In [116]:
get_address_json('Солнечногорск улица Дзержинского, 19')

,address,lat,lon,country,province,area,locality,street,house,precision
0,"Россия, Московская область, Солнечногорск, ули...",56.185763,36.983604,Россия,Московская область,городской округ Солнечногорск,Солнечногорск,улица Дзержинского,19,exact


In [117]:
get_address_json('30.313263; 59.930317')

,address,lat,lon,country,province,locality,street,house,precision
0,"Россия, Санкт-Петербург, переулок Гривцова, 5В",59.93034,30.313532,Россия,Санкт-Петербург,Санкт-Петербург,переулок Гривцова,5В,exact


In [118]:
get_address_json('59.930317, 30.313263')

,address,lat,lon,country,province,precision
0,"Иран, Систан и Белуджистан",27.687912,60.721631,Иран,Систан и Белуджистан,other
